In [1]:
from qaoa_and_tdvp import QAOA, tdvp_optimize_qaoa, scipy_optimize, QAOAResult, Qobj
from MaxCut import MaxCut
from qutip import tensor
from qutip.qip.operations import expand_operator, rz
from benchmark import get_all_connected, get_connected_rn_graph, get_rn_qubo, select_if_connected, Benchmark
from qutip.parallel import parallel_map, serial_map
from itertools import combinations_with_replacement
from multiprocessing import Value
import pickle
import numpy as np

In [2]:
p = 2
n=4
seed = 42
with open(f"./instances/n{n}_graphs.p","rb") as f:
    graphs = pickle.load(f)

In [3]:
index = 5#np.random.randint(0,len(graphs)); print(index)

In [4]:
instance = MaxCut(graphs[index], draw=False)
qaoa = QAOA(qubo=instance.qubo, p=p, mapping=serial_map)
delta = tuple(1 for _ in range(2 * qaoa.p))

In [5]:
qaoa.state(delta)

Quantum object: dims = [[2, 2, 2, 2], [1, 1, 1, 1]], shape = (16, 1), type = ket
Qobj data =
[[-0.46066168+0.27994601j]
 [-0.01209181-0.23061102j]
 [-0.01209181-0.23061102j]
 [ 0.10950453+0.05633505j]
 [-0.01209181-0.23061102j]
 [ 0.10950453+0.05633505j]
 [ 0.10950453+0.05633505j]
 [-0.07620785-0.10970592j]
 [-0.01209181-0.23061102j]
 [ 0.10950453+0.05633505j]
 [ 0.10950453+0.05633505j]
 [-0.07620785-0.10970592j]
 [ 0.10950453+0.05633505j]
 [-0.07620785-0.10970592j]
 [-0.07620785-0.10970592j]
 [-0.32599319+0.47694602j]]

In [6]:
qaoa.gram(delta)

matrix([[ 11.47806041+2.22044605e-16j,   5.87147831-3.16333014e+00j,
          -2.55200695-1.39043924e-01j,  -2.67512163+4.08226904e-01j],
        [  5.87147831+3.16333014e+00j,   6.6505476 +2.13717932e-15j,
          -2.25473696+1.08072763e+00j,  -2.70210342+2.03083440e+00j],
        [ -2.55200695+1.39043924e-01j,  -2.25473696-1.08072763e+00j,
          36.75      +0.00000000e+00j, -53.10865499+2.28984222e+01j],
        [ -2.67512163-4.08226904e-01j,  -2.70210342-2.03083440e+00j,
         -53.10865499-2.28984222e+01j, 114.62047923-6.34561848e-15j]])

In [ ]:
res = tdvp_optimize_qaoa(
    qaoa,
    delta,
    1,
    int_mode="RK45",
    rhs_mode="qaoa",
    grad_tol=0.1,
    max_iter=300,
)
print(res)

In [41]:
sci_res = scipy_optimize(qaoa, delta)
sci_res


        scipy_cobyla terminated with sucess with message
        "Optimization terminated successfully."
        This took 5.25 seconds
        
            optimal parameters: [1.56733882 0.78449255 1.82667484 1.07586705]
                 optimal value: 17.261663823713175
        maximal ground overlap: 0.4518900748808219
           number of fun calls: 260
               number of steps: None
               number of gates: 7280
        